# Scoping

In [ ]:
# Financial Advising for Farmers, hired to to advise.
# Where is the best area in the praires to invest in a farm with a rotation crop with mainly barley, and spring wheat


# ETL (Extract, Transform, Load)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import seaborn as sb
plt.rcParams['figure.figsize'] = [12, 8]

# Yield

In [ ]:
df_sk=pd.read_csv("C:/Users/prisc/Desktop/AG/Python Data Science/Project/Data/rm-yields-data.csv")

In [ ]:
df_mb=pd.read_excel("C:/Users/prisc/Desktop/AG/Python Data Science/Project/Data/MMPP - Yield by Soil Type Browser.xlsx")

# Shapefiles

In [ ]:
gdf_sk = gpd.read_file("C:/Users/prisc/Desktop/AG/Python Data Science/Project/Data/RuralMunicipality.shp")

In [ ]:
gdf_mb = gpd.read_file("C:/Users/prisc/Desktop/AG/Python Data Science/Project/Data/MB_Municipal_Boundaries.shp")

# Transforming

In [ ]:
df_mb['Yield/acre(Metric)']= df_mb['Yield/acre(Metric)'].astype(str).str.replace(' Tonnes', '') # Replacing Tonnes
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].replace('Tolerance', np.NaN) # Replacing Tolerance
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].astype(float) # changing object to float data type


In [ ]:
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('RM OF ','')
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('RM OF ','')
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('MUNICIPALITY OF ','', regex=False) # not caps sensitive\n",
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace(' MUNICIPALITY','', regex=False) # not caps sensitive\n",
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('CITY OF ','', regex=False) # not caps sensitive"
   

In [ ]:
df_mb_pivot=pd.pivot_table(df_mb.drop(columns=['Yield/acre(Metric).1', 'Yield/acre(Imperial)', 'Soil', 'Farms' ]),
               index=['Risk Area / R.M.', 'Year'], columns='Crop', values='Yield/acre(Metric)')\
               .reset_index()

In [ ]:
df_mb_clean=df_mb_pivot.rename(columns={
    'Risk Area / R.M.': 'RM', 
    'ARGENTINE CANOLA': 'Canola', 
    'BARLEY':'Barley',
    'CANARYSEED': 'Canary Seed', 
    'DURUM WHEAT': 'Durum Wheat', 
     'LENTILS': 'Lentils', 
     'OATS': 'Oats',
    'RED SPRING WHEAT': 'Spring Wheat', 
    'WHITE PEA BEANS': 'Peas'}) \
        .drop(columns=['ALFALFA', 'FABABEANS', 'FABABEANS', 'POLISH CANOLA'] )

In [ ]:
df_sk_clean=df_sk.rename(columns={'Durum': 'Durum Wheat'}).drop(columns=['Winter Wheat', 'Mustard', 'Sunflowers', 'Fall Rye', 'Spring Rye', 'Tame Hay','Flax', 'Chickpeas' ] )

In [ ]:
# Crop conversion in MB tonnes to bushel
df_mb_clean['Canola']=df_mb_clean['Canola'] * 44.092
df_mb_clean['Barley']=df_mb_clean['Barley'] * 45.93
df_mb_clean['Canary Seed']=df_mb_clean['Canary Seed'] * 44.092
df_mb_clean['Durum Wheat']=df_mb_clean['Durum Wheat'] * 36.74
df_mb_clean['Lentils']=df_mb_clean['Lentils'] * 36.74
df_mb_clean['Oats']=df_mb_clean['Oats'] * 64.842
df_mb_clean['Spring Wheat']=df_mb_clean['Spring Wheat'] * 36.74
df_mb_clean['Peas']=df_mb_clean['Peas'] *  36.74

In [ ]:
# Crop Conversion in SK - pounds to bushels
df_sk_clean['Lentils']=df_sk_clean['Lentils'] / 60
df_sk_clean['Canary Seed']=df_sk_clean['Canary Seed'] / 50

In [ ]:
# Creating province column
df_mb_clean['Province'] = 'MB'
df_sk_clean['Province'] = 'SK'

In [ ]:
df = pd.concat([df_mb_clean, df_sk_clean])

In [ ]:
df.head()
unique_rm_values = df['RM'].nunique()

print("Number of unique values in RM:", unique_rm_values)

# EDA(Exploratory Data Analysis)

# Missing Values

In [ ]:
df.loc[df['Year']>2000].isna().sum().sort_values().plot(kind='bar', color='blue')
plt.title('Missing Values - 2002 to 2021')
plt.xlabel('Crops')
plt.ylabel('# of Missing values')
plt.axhline(len(df)/2, linestyle='--', color='red')
plt.axhline(len(df)/4, linestyle='--', color='red')
plt.axhline(len(df)/10, linestyle='--', color='red')
plt.show()

# Outliers

# Histograms

In [ ]:
crop_columns=['Barley', 'Spring Wheat']

In [ ]:
df[crop_columns].hist(bins=300)
plt.show()

In [ ]:
# Peorson Corr (-1 to 1), -1 negative corr, o no corr, 1 positive corr
# Using Seaborn
sb.heatmap(df[crop_columns].corr(),annot=True, cmap='Greens')

# Boxplots

In [ ]:
def find_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return column[(column < lower_bound) | (column > upper_bound)]

# Find outliers for "Barley" and "Spring Wheat"
barley_outliers = find_outliers(df['Barley'])
spring_wheat_outliers = find_outliers(df['Spring Wheat'])

print("Outliers for Barley:")
print(barley_outliers)

print("\nOutliers for Spring Wheat:")
print(spring_wheat_outliers)

In [ ]:
df[crop_columns].boxplot()

In [ ]:
df.loc[df['Spring Wheat']>180]

In [ ]:
df.drop(df[df['RM'] == 314].index, inplace=True)

# GIS Analyis

In [ ]:
# CRS --> Cordinate Reference Systems
gdf_mb['geometry'].crs

In [ ]:
# Standardazing CRS formats
gdf_sk['geometry']=gdf_sk['geometry'].to_crs('epsg:4326')
gdf_mb['geometry']=gdf_mb['geometry'].to_crs('epsg:4326')

In [ ]:
# Renaming column name
gdf_sk['RMNO']=gdf_sk['RMNO'].astype(int)


In [ ]:
gdf_mb['MUNI_NAME']=gdf_mb['MUNI_NAME'].str.replace('RM OF ','')

In [ ]:
gdf=pd.concat([gdf_sk[['RMNO','geometry']].rename(columns={'RMNO':'RM'}),gdf_mb[['MUNI_NAME', 'geometry']].rename(columns={'MUNI_NAME':'RM'})])

In [ ]:
gdf['RM']=gdf['RM'].astype('string')
df['RM']=df['RM'].astype('string')

In [ ]:
gdf.plot()

In [ ]:
# SQL inner join
df_gdf=pd.merge(gdf, df, on='RM', how='inner') 

In [ ]:
print('Before merging', gdf['RM'].nunique())
print('After merging',df_gdf['RM'].nunique())

In [ ]:
ds=df_gdf.copy()

# GIS Visualization

In [ ]:
ds.loc[ds['Year']>=2000].plot(column='Barley', 
                              legend=True,
                              cmap='Greens', 
                              scheme="quantiles",
                              figsize=(15, 10),
                              missing_kwds={ "color": "lightgrey",
                                            "edgecolor": "red",
                                            "hatch": "///",
                                            "label": "Missing values",
                                            },
                              
                              );

In [ ]:
ds.loc[ds['Year']>=2000].plot(column='Spring Wheat', 
                              legend=True, 
                              cmap='OrRd', 
                              scheme="quantiles",
                              figsize=(15, 10),
                              missing_kwds={ "color": "lightgrey",
                                            "edgecolor": "red",
                                            "hatch": "///",
                                            "label": "Missing values",
                                            },
                              
                              );

# Agregations

In [ ]:
df_12_21=df.loc[df['Year']>2011].sort_values(['RM', 'Year'])

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(15, 20))
years = df_12_21['Year'].unique()

for i, year in enumerate(years):
    if i<10:
        ax = axes[i // 2, i % 2]
        merged_df = pd.merge(gdf, df_12_21.loc[df_12_21['Year'] == year], on='RM')
        merged_df.plot(column='Barley', cmap='RdYlGn', legend=True, ax=ax)
        ax.set_title(f' Barley in {year}', color='Black', size=12)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(15, 20))
years = df_12_21['Year'].unique()

for i, year in enumerate(years):
    if i<10:
        ax = axes[i // 2, i % 2]
        merged_df = pd.merge(gdf, df_12_21.loc[df_12_21['Year'] == year], on='RM')
        merged_df.plot(column='Spring Wheat', cmap='RdYlGn', legend=True, ax=ax)
        ax.set_title(f' Spring Wheat Yield in {year}', color='Black', size=12)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate the average 5 most productable cities for barley, and spring wheat in the last 20 years.

current_year = pd.Timestamp.now().year  # Get the current year
start_year = current_year - 19  # Calculate the starting year for the last 20 years

filtered_data = df[df['Year'] >= start_year]

# Group the data by the RM column and calculate the mean for specific columns
columns_to_average = ['Barley', 'Spring Wheat',]
result = filtered_data.groupby(['RM', 'Province'])[columns_to_average].mean().reset_index()

result_sorted_barley = result.sort_values('Barley', ascending=False)
result_sorted_wheat = result.sort_values('Spring Wheat', ascending=False)

top_5_barley = result_sorted_barley.head(5)
top_5_barley = top_5_barley.drop('Spring Wheat', axis = 1)
top_5_spring_wheat = result_sorted_wheat.head(5)
top_5_spring_wheat = top_5_spring_wheat.drop('Barley', axis = 1)

print(top_5_spring_wheat)
print(top_5_barley)



In [ ]:
#Create the estimative revenue for these cities
barley_price = 6.86 #Average price barley per bushel in 2021
spring_wheat_price = 9.76 #Average price spring_wheat (https://afsc.ca/wp-content/uploads/2021/06/2021-Feed-Grain.pdf)

top_5_barley['Estimated_Revenue'] = barley_price * top_5_barley['Barley']
top_5_spring_wheat['Estimated_Revenue'] = spring_wheat_price * top_5_spring_wheat['Spring Wheat']

print (top_5_barley)
print (top_5_spring_wheat)


In [ ]:
# Create bar graphs for the top 5 cities for Barley and Spring Wheat
plt.figure(figsize=(12, 6))
plt.subplot(121)  # Bar graph for Barley
bar_width = 0.7

plt.bar(top_5_barley['RM'] + " / " + top_5_barley['Province'] , top_5_barley['Barley'], width = bar_width)
plt.title('Top 5 Cities for Barley (Last 20 Years)')
plt.xlabel('City (Province)')
plt.ylabel('Average Yield for Barley')
plt.xticks(rotation=45)  # Rotate x-axis labels by 45 degrees

plt.subplot(122)  # Bar graph for Spring Wheat
plt.bar(top_5_spring_wheat['RM']+ " / " + top_5_spring_wheat['Province'], top_5_spring_wheat['Spring Wheat'], width = bar_width)
plt.title('Top 5 Cities for Spring Wheat (Last 20 Years)')
plt.xlabel('City (Province)')
plt.ylabel('Average Yield for Spring Wheat')
plt.xticks(rotation=45)  # Rotate x-axis labels by 45 degrees

plt.tight_layout()
plt.show()


In [ ]:
from matplotlib.ticker import FormatStrFormatter
# Filter the data for the RM 'Louise' and starting year 2002
rm_mac_data = df[(df['RM'] == 'LOUISE') & (df['Year'] >= 2001)]
# Convert the "Year" column to integers
fig, ax = plt.subplots()
ax.plot(rm_mac_data['Year'], rm_mac_data['Barley'], marker='o', linestyle='-', label='Barley')
ax.plot(rm_mac_data['Year'], rm_mac_data['Spring Wheat'], marker='o', linestyle='--', label='Spring Wheat')
plt.title('Production of Barley and Spring Wheat in Louise Over the Years')
plt.ylabel('Production')
plt.xlabel('Year')
ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
plt.grid()
plt.show()

In [ ]:
agg_funcs = {
    column: ['mean', 'std', 'median'] for column in crop_columns
}

df_agg = df_12_21.set_index('RM')[crop_columns].groupby('RM').agg(agg_funcs)

# Optionally, to flatten the multi-level columns:
df_agg.columns = ['_'.join(col).strip() for col in df_agg.columns.values]

# Dropping missing values for Barley mean.
df_agg=df_agg.dropna(subset='Barley_mean')
df_agg=df_agg.dropna(subset='Spring Wheat_mean')
df_agg=df_agg.dropna(subset='Barley_std')
df_agg=df_agg.dropna(subset='Spring Wheat_std')
df_agg.to_csv ('/Users/prisc/Desktop/stats.csv')


# Feature Selection

In [ ]:
import os

# Set OMP_NUM_THREADS to 2 to avoid the memory leak issue
os.environ['OMP_NUM_THREADS'] = "2"

# Importing library
from sklearn.cluster import KMeans 

X = df_agg.loc[:,['Barley_mean', 'Barley_std', 'Spring Wheat_mean', 'Spring Wheat_std']]

from sklearn.metrics import silhouette_score
n_clusters = [2,3,4,5,6,7,8,9,10,11,12,13,14,15] # number of clusters
clusters_inertia = [] # inertia of clusters
s_scores = [] # silhouette scores

for n in n_clusters:
    KM = KMeans(n_clusters=n, init='k-means++', n_init=10).fit(X)
    clusters_inertia.append(KM.inertia_)    # data for the elbow method
    silhouette_avg = silhouette_score(X, KM.labels_)
    s_scores.append(silhouette_avg) # data for the silhouette score method


In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(x= X['Barley_mean'], y=X['Barley_std'])
plt.xlabel('Barley_mean')
plt.ylabel('Barley_std');

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(x= X['Spring Wheat_mean'], y=X['Spring Wheat_std'])
plt.xlabel('Spring Wheat_mean')
plt.ylabel('Spring Wheat_std');

# Elbow Method

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(n_clusters, clusters_inertia, 'o-', color='blue', label='Elbow Method')
ax.set_title("Elbow Method")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Clusters Inertia")
ax.axvline(6, ls="--", c="red")
plt.grid()
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(n_clusters, s_scores, 's-', color='green', label='Silhouette Score Method')
ax.set_title("Silhouette Score Method")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Silhouette Score")
ax.axvline(6, ls="--", c="red")
plt.grid()
plt.legend()
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=6, init='k-means++', random_state=42, n_init=10)
X['Clusters_6']=kmeans.fit_predict(X)

In [ ]:
custom_palette = ["red", "green", "blue", "orange", "purple", "pink"]
sb.scatterplot(data=X, x='Barley_mean', y='Barley_std', hue='Clusters_6', palette=custom_palette)
plt.title('Barley Clustering Mean and Std | 2003-2022 | K-Means ', size =14)
plt.show()

In [ ]:
custom_palette = ["magenta", "olive", "brown", "yellow", "black", "grey"]
sb.scatterplot(data=X, x='Spring Wheat_mean', y='Spring Wheat_std', hue='Clusters_6', palette=custom_palette)
plt.title('Spring Wheat Clustering Mean and Std | 2003-2022 | K-Means ', size =14)
plt.show()

In [ ]:
X.groupby('Clusters_6').mean()\
    .sort_values('Barley_mean')[['Barley_mean',	'Barley_std']]

In [ ]:
# Ranking based on the mean Barley
X['Clusters_6_ranked']=X['Clusters_6'].replace(to_replace={
    5:0,
    4:1,
    0:2,
    2:3,
    3:4,
    1:5,
})

In [ ]:
pd.merge(
    gdf,
    X,
    on='RM'
).explore(column='Clusters_6_ranked', legend='True', k=6, scheme='naturalbreaks', cmap='Oranges')

In [ ]:
X.groupby('Clusters_6').mean().sort_values('Spring Wheat_mean')[['Spring Wheat_mean',	'Spring Wheat_std']]

In [ ]:
# Ranking based on the mean
X['Clusters_6_ranked']=X['Clusters_6'].replace(to_replace={
    5:0,
    4:1,
    0:2,
    2:3,
    3:4,
    1:5,
})

In [ ]:
pd.merge(
    gdf,
    X,
    on='RM'
).explore(column='Clusters_6_ranked', legend='True', k=6, scheme='naturalbreaks', cmap='Purples')